In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/skku/2021-1 Data Mining/dm-text-analysis/polarity.csv')
df

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
...,...,...,...,...,...,...,...,...,...
16357,힘들/VA;ㄹ/ETM;것/NNB,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16358,힘들/VA;ㄹ/ETM;때/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16359,힘차/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16360,힘차/VA;ㄴ/ETM,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [3]:
lala_review = pd.read_csv('/content/drive/MyDrive/skku/2021-1 Data Mining/dm-text-analysis/naver_preprocess.csv',encoding='utf-8')
lala_review.head()

,score,text,idx,user,written_at,agree,disagree,body
0,10,극장에서 3번이나 봤습니다 그래도 보고싶은 너무나 행복한 영화입니다,12180460,후니뷘(beau****),2017.01.15 17:00,1,1,극장에서 3번이나 봤습니다 그래도 보고 싶은 너무나 행복한 영화입니다
1,10,꿈의나라로~!!!!,12180563,더리버리(rapp****),2017.01.15 17:16,2,1,꿈의 나라로
2,9,반전과 공감이 있는 로맨스 영화,12180612,조윤경(jykj****),2017.01.15 17:26,2,1,반전과 공감이 있는 로맨스 영화
3,8,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이였지만 그렇지 못해서 더 애잔함...,12180615,상하(shtb****),2017.01.15 17:27,2,0,마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이었지만 그렇지 못해서 더 애잔함...
4,8,영상미는 진짜 깜짝 놀랄정도였다근데 난 개인적으로 여주가 극적으로 성공하는게 너무 ...,12180617,이소의(thdm****),2017.01.15 17:27,2,0,영상미는 진짜 깜짝 놀랄 정도였다 근데 난 개인적으로 여주가 극적으로 성공하는 게 ...


In [4]:
lala_review = lala_review.dropna()

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python-dev #파이썬 버전 맞춰서 
pip install JPype1
pip install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/

In [6]:
from konlpy.tag import Okt
tokenizer = Okt()

In [27]:
lala_review.body.iloc[1]

'꿈의 나라로'

In [8]:
tokenizer.morphs(lala_review.body.iloc[0])

['극장',
 '에서',
 '3',
 '번',
 '이나',
 '봤습니다',
 '그래도',
 '보고',
 '싶은',
 '너무나',
 '행복한',
 '영화',
 '입니다']

In [9]:
tmp = tokenizer.pos(lala_review.body.iloc[0],stem=True)
tmp

[('극장', 'Noun'),
 ('에서', 'Josa'),
 ('3', 'Number'),
 ('번', 'Noun'),
 ('이나', 'Josa'),
 ('보다', 'Verb'),
 ('그래도', 'Adverb'),
 ('보고', 'Noun'),
 ('싶다', 'Verb'),
 ('너무나', 'Adverb'),
 ('행복하다', 'Adjective'),
 ('영화', 'Noun'),
 ('이다', 'Adjective')]

In [33]:
def word_class_tag(tag):
    """
    Convert between the word class tags to simple tags
    """
    if tag == 'Noun':
        return 'N'
    elif tag == 'Verb':
        return 'VV'
    elif tag == 'Adverb':
        return 'AV'
    elif tag == 'Adjective':
        return 'AD'
    elif tag == 'Josa': #조사
        return 'J'
    elif tag == 'VerbPrefix' or tag == 'Suffix': #접두사 접미사
        return 'X'
    elif tag == 'Number':
        return 'S'
    elif tag == 'Modifier' or tag == 'Conjunction' or tag == 'Determiner': #관형사 부사
        return 'M'
    elif tag == 'Exclamation': #감탄사
        return 'I'
    return None
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
    sentiment = 0.0
    tokens_count = 0

    raw_sentences = tokenizer.pos(text,stem=True)
    for raw_sentence in raw_sentences:
        if word_class_tag(raw_sentence[1]) != None:
            df_raw = df.loc[df['ngram'].str.contains(raw_sentence[0]) & df['ngram'].str.contains(word_class_tag(raw_sentence[1]))]
            ngram = df_raw['ngram'].tolist()
            ngram = [n[:len(raw_sentence[0])] for n in ngram]
            index = 0
            for i in range(len(ngram)):
                if ngram[i] == raw_sentence[0]:
                    index = i
                    break
            sentiment += df['POS'][index] - df['NEG'][index]
        tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0

In [29]:
for index, (text, score) in enumerate(zip(lala_review.iloc[:4].body, lala_review.iloc[:4].score)):
    print(f'Text {index} : {text.strip()}')
    sentiment = "positive" if score > 5 else "negative"
    print('Sentiment:', sentiment, score)
    print('Predicted Sentiment polarity:', swn_polarity(text))
    print('-'*60)

Text 0 : 극장에서 3번이나 봤습니다 그래도 보고 싶은 너무나 행복한 영화입니다
Sentiment: positive 10
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 1 : 꿈의 나라로
Sentiment: positive 10
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 2 : 반전과 공감이 있는 로맨스 영화
Sentiment: positive 9
Predicted Sentiment polarity: 1
------------------------------------------------------------
Text 3 : 마지막에 남녀가 만나서 사랑을 이뤘으면 행복한 결말이었지만 그렇지 못해서 더 애잔함이 묻어나는 아름다운 영화
Sentiment: positive 8
Predicted Sentiment polarity: 1
------------------------------------------------------------


In [30]:
y_true, y_pred = [], []
for index, (text, score) in enumerate(zip(lala_review.iloc[:].body, lala_review.iloc[:].score)):
    true = 1 if score > 5 else 0
    y_true.append(true)
    pred = swn_polarity(text)
    y_pred.append(pred)

In [31]:
print(y_true)
print(y_pred)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [32]:
from sklearn.metrics import classification_report
target_names = ['negative', 'positive']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.10      0.01      0.02        77
    positive       0.92      0.99      0.96       914

    accuracy                           0.91       991
   macro avg       0.51      0.50      0.49       991
weighted avg       0.86      0.91      0.88       991

